In [1]:
import numpy as np
import matplotlib.pyplot as plt
import functions
from pprint import pprint
import json
from scipy import interpolate
import langmuir.gymness.aggregate as agg
%matplotlib

Using matplotlib backend: Qt5Agg


In [ ]:
# definisco i file
lamp_file = 'OES_Studenti_2017/Ilamp_acquisitiontime_3s.txt'
rho_file = 'OES_Studenti_2017/riflettivitadiffusore.txt' 
argon_750_file = 'OES_Studenti_2017/Ar15022017_a8_acq_time_0.01s.txt' 
argon_480_file = 'OES_Studenti_2017/Ar15022017_a1_acq_time_0.5s.txt' 

acquisition_time = 0.01

peak_ratio = functions.get_peak_ratio(argon_480_file)
[I_abs_spectrum, n_0, I_abs_750, X_750] = functions.get_absolute_intensity(lamp_file, rho_file, argon_750_file, acquisition_time)

In [ ]:
print("""
Peak ratio I480/I488 = {}
n0 = {} cm-3
I absolute 750 = {} 
X 750 = {}
""". format(peak_ratio, n_0, I_abs_750, X_750))

In [ ]:
# first pression spectra analysys
#                                                    name of config file    folder with data
ratios, w_em, results = functions.get_profile_ratio('config_p1_2302.json', '2302/2302spectr/')
n_avg, n_min, n_max   = functions.get_density(ratios)
# plotting data
fig_spectr, ax_spectr = plt.subplots(1,1)
ax_spectr.errorbar(w_em, n_avg, yerr=n_max-n_min, fmt='o')
ax_spectr.grid()

In [270]:
# langmuir density and profile temperature
#                        path to results data
df = agg.aggregate_data('langmuir/data/data-2302')
# the grouped by columns will become indexes
a = df.groupby(['P_off', 'W_em'])['n_e', 'err_n_e', 'T_e', 'err_T_e'].mean()
# set plot stles, titles and figures. Need to add the labels though
fig = plt.figure()
ax_ne = fig.add_subplot(211)
ax_ne.set_title('$n_e$ $cm^{-3}$')
ax_ne.grid()
ax_te = fig.add_subplot(212)
ax_te.set_title('$T_e$ (eV)')
ax_te.grid()
plt.tight_layout()  # this is useful for overlapping figures

# difficult code: iterate over the values of the pressure
# indexes (there are just two values in this case) and
# take the values of the w, n, and T for each p.
# Then plot all the data with their labels
for p in a.index.get_level_values(0).unique():
    w = a.index.get_level_values(1).unique()
    n_e = a.loc[p,:]['n_e']
    err_n_e = a.loc[p,:]['err_n_e']
    T_e = a.loc[p,:]['T_e']
    err_T_e = a.loc[p,:]['err_T_e']
    ax_ne.errorbar(w, n_e, yerr=err_n_e, fmt='o', label="p: {} mbar".format(p))
    ax_te.errorbar(w, T_e, yerr=err_T_e, fmt='o', label="p: {} mbar".format(p))
    
# set legend for the plots
ax_ne.legend()
ax_te.legend()

Reading dataset:['2302lang10', '2302lang09', '2302lang03', '2302lang11', '2302lang04', '2302lang06', '2302lang05', '2302lang08', '2302lang02', '2302lang07', '2302lang12', '2302lang01']
